In [1]:
import jieba
import pandas as pd
import re
import pickle
from datetime import datetime
import seaborn as sns 
import matplotlib.pyplot as plt
import unicodedata # for removing Chinese puctuation


In [11]:
df = pd.read_csv("./芷瑩候選人/output-clean剩下五縣市-output.csv")[['PageName', "Message"]]
df.head(2)

,PageName,Message
0,伍麗華｜Saidai / Reseres,天啊，竟然有這樣的結果？！ 眾目睽睽下，進行不記名投票，完全輕看在地居民，真是不可思議！ 今...
1,伍麗華｜Saidai / Reseres,很多人沒聽過「大後」，就像沒聽過「小路」。 我是因為去了泰武鄉服務，才知道來義鄉有個大後部落...


# tokenization


In [12]:
# 停用詞
with open("./stop_word_chinese.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[1:]
print(stopwords[:10])
print(len(stopwords))

['\\u3000', '?', '、', '。', '“', '”', '，', '。', '！', '？']
927


In [13]:
#新定義辭典d
with open("./add_word_healthDoctor.txt", encoding="utf-8") as f:
    add_newword = f.read().split("\n")

for word in add_newword: jieba.add_word(word)
    
with open("./dict.txt.big", encoding="utf-8") as f:
    add_newword = f.read().split("\n")


In [14]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # 表情符號 (Emoticons)
                               u"\U0001F300-\U0001F5FF"  # 圖形符號 (Miscellaneous Symbols and Pictographs)
                               u"\U0001F680-\U0001F6FF"  # 交通運輸符號 (Transport and Map Symbols)
                               u"\U0001F1E0-\U0001F1FF"  # 國旗符號 (Flags)
                               u"\U00002600-\U000027BF"  # 雜項符號 (Miscellaneous Symbols)
                               u"\U0001f926-\U0001f937"  # 表情符號 (Emoticons)
                               u"\U00010000-\U0010FFFF"  # 其他符號 (Other Symbols)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [15]:
text_with_emoji = "這是一個包含😃 emoji的文本。"
remove_emoji(text_with_emoji)

'這是一個包含 emoji的文本。'

In [16]:

df['Message'] = df['Message'].apply(lambda x:re.sub(r"http\S+", "", x))


In [17]:
df['Message'] = df['Message'].apply(lambda x:(x.replace("\u3000", "")))

In [18]:
df['Message'] = df['Message'].apply(lambda x:(remove_emoji(x)))



In [19]:
df['Message'].apply(lambda x:(x.replace("\u3000", "")))

0       天啊，竟然有這樣的結果？！ 眾目睽睽下，進行不記名投票，完全輕看在地居民，真是不可思議！ 今...
1       很多人沒聽過「大後」，就像沒聽過「小路」。 我是因為去了泰武鄉服務，才知道來義鄉有個大後部落...
2       93軍人節，伍麗華帶您一起認識原住民最高軍階將領！ 原住民在部隊，一直是特戰單位的主力，原住...
3       颱風天，「風雨故人來」，按字面意思，風雨天，有個朋友來看你，與你坐下來敘談。它寫的是一種心境...
4       海端鄉最裡面的利稻村，昨天颱風登陸後，電力中斷、道路坍方。有村民走了2公里，拍回來坍方路況。...
                              ...                        
3274    【團圓】 大家吃湯圓了嗎？ 身為一個民意代表，在外的時間比在家多。 秀芳今天努力跑跑跑，到了...
3275    【環島拼圖】 賴蕭配「環島拼圖」活動，今天來到彰化 二林至揚宮！ 賴清德總統候選人登台時，原...
3276    【一起推廣CPR】 秀芳下午和 DPAC 彰化縣防災協會 合辦「全民救護推廣」，也舉辦「彰化...
3277    【小英總統來了】選對的人，走對的路 小英總統 蔡英文 Tsai Ing-wen 今天來到我們...
3278    【水牛伯】立法院長游錫堃 感謝立法院長 游游錫堃傍晚到彰化芬園 好好漢將軍廟（芬園。社口）為...
Name: Message, Length: 3279, dtype: object

In [20]:
df.head(2)

,PageName,Message
0,伍麗華｜Saidai / Reseres,天啊，竟然有這樣的結果？！ 眾目睽睽下，進行不記名投票，完全輕看在地居民，真是不可思議！ 今...
1,伍麗華｜Saidai / Reseres,很多人沒聽過「大後」，就像沒聽過「小路」。 我是因為去了泰武鄉服務，才知道來義鄉有個大後部落...


# 斷詞

In [21]:
# 斷詞ing
df['tokenized'] = df['Message'].apply(lambda x:list(jieba.cut(x)))

In [22]:
df.head(1)

,PageName,Message,tokenized
0,伍麗華｜Saidai / Reseres,天啊，竟然有這樣的結果？！ 眾目睽睽下，進行不記名投票，完全輕看在地居民，真是不可思議！ 今...,"[天, 啊, ，, 竟然, 有, 這樣, 的, 結果, ？, ！, , 眾目, 睽睽, 下..."


In [23]:
def remove_stopWords(words):
    out = []
    for word in words:
        if word not in stopwords:
            out.append(word)
    return out

def remove_punc_by_unicode(words):
    out = []
    for word in words:
        if word != " " and not unicodedata.category(word[0]).startswith('P'):
            out.append(word)
    return out

In [24]:
df['cleaned'] = df['tokenized'].apply(remove_stopWords)
df['cleaned_removePunc'] = df['cleaned'].apply(remove_punc_by_unicode)



In [25]:
df['cleaned_removePunc'] = df['cleaned_removePunc'].apply(lambda x: [item for item in x if item != "" and len(item) >= 2 and len(item) <= 15])


In [26]:
df.tail(10)

,PageName,Message,tokenized,cleaned,cleaned_removePunc
3269,黃秀芳,【聖誕節】 選戰倒數ing，2023也悄悄倒數了！ 秀芳下午出席彰化縣音樂河社會福利協會舉辦...,"[【, 聖誕節, 】, , 選戰, 倒數, ing, ，, 2023, 也, 悄悄, 倒數...","[聖誕節, , 選戰, 倒數, ing, 2023, 悄悄, 倒數, , 秀芳, 下午,...","[聖誕節, 選戰, 倒數, ing, 2023, 悄悄, 倒數, 秀芳, 下午, 出席, 彰..."
3270,黃秀芳,【鄭家純來了】免費參加，一起推廣 半夜暗著來的！名額有限！ 寒夜還沒睡的您！搶先看到！速速手...,"[【, 鄭家, 純來, 了, 】, 免費, 參加, ，, 一起, 推廣, , 半夜, 暗著...","[鄭家, 純來, 免費, 參加, 推廣, , 半夜, 暗著來, 名額, 有限, , 寒夜...","[鄭家, 純來, 免費, 參加, 推廣, 半夜, 暗著來, 名額, 有限, 寒夜, 還沒, ..."
3271,黃秀芳,【1號黃秀芳 懇請疼惜】 秀芳上午抽到1號，喊出「信賴有芳，支持1號黃秀芳」！ 從事政治運動...,"[【, 1, 號黃秀芳, , 懇請, 疼惜, 】, , 秀芳, 上午, 抽到, 1, 號...","[1, 號黃秀芳, , 懇請, 疼惜, , 秀芳, 上午, 抽到, 1, 號, 喊出, ...","[號黃秀芳, 懇請, 疼惜, 秀芳, 上午, 抽到, 喊出, 信賴, 有芳, 支持, 號黃秀..."
3272,黃秀芳,【溫暖的擁抱】 傍晚的彰化，寒風凜冽，新北市議員 山田摩衣，特地來到彰化，陪著秀芳挨家挨戶掃...,"[【, 溫暖, 的, 擁抱, 】, , 傍晚, 的, 彰化, ，, 寒風凜冽, ，, 新北...","[溫暖, 擁抱, , 傍晚, 彰化, 寒風凜冽, 新北市, 議員, , 山田, 摩衣, ...","[溫暖, 擁抱, 傍晚, 彰化, 寒風凜冽, 新北市, 議員, 山田, 摩衣, 特地, 彰化..."
3273,黃秀芳,【把握黃金周】黃秀芳全力以赴 選前剩下3個黃金周末，這周六(12/23)上午，賴蕭配「唱土地...,"[【, 把握, 黃, 金周, 】, 黃, 秀芳, 全力以赴, , 選前, 剩下, 3, 個...","[把握, 黃, 金周, 黃, 秀芳, 全力以赴, , 選前, 剩下, 3, 黃, 金, 周...","[把握, 金周, 秀芳, 全力以赴, 選前, 剩下, 周末, 周六, 12, 23, 上午,..."
3274,黃秀芳,【團圓】 大家吃湯圓了嗎？ 身為一個民意代表，在外的時間比在家多。 秀芳今天努力跑跑跑，到了...,"[【, 團圓, 】, , 大家, 吃, 湯圓, 了, 嗎, ？, , 身為, 一個, 民...","[團圓, , 吃, 湯圓, , 身為, 一個, 民意代表, 在外, 時間, 在家, ,...","[團圓, 湯圓, 身為, 一個, 民意代表, 在外, 時間, 在家, 秀芳, 努力, 跑跑,..."
3275,黃秀芳,【環島拼圖】 賴蕭配「環島拼圖」活動，今天來到彰化 二林至揚宮！ 賴清德總統候選人登台時，原...,"[【, 環島, 拼圖, 】, , 賴蕭配, 「, 環島, 拼圖, 」, 活動, ，, 今天...","[環島, 拼圖, , 賴蕭配, 環島, 拼圖, 活動, 彰化, , 二林, 至揚宮, ...","[環島, 拼圖, 賴蕭配, 環島, 拼圖, 活動, 彰化, 二林, 至揚宮, 賴清德, 總統..."
3276,黃秀芳,【一起推廣CPR】 秀芳下午和 DPAC 彰化縣防災協會 合辦「全民救護推廣」，也舉辦「彰化...,"[【, 一起, 推廣, CPR, 】, , 秀芳, 下午, 和, , DPAC, , ...","[推廣, CPR, , 秀芳, 下午, , DPAC, , 彰化, 縣防災, 協會, ...","[推廣, CPR, 秀芳, 下午, DPAC, 彰化, 縣防災, 協會, 合辦, 全民, 救..."
3277,黃秀芳,【小英總統來了】選對的人，走對的路 小英總統 蔡英文 Tsai Ing-wen 今天來到我們...,"[【, 小英, 總統, 來, 了, 】, 選對, 的, 人, ，, 走, 對, 的, 路, ...","[小英, 總統, 選對, 人, 走, 對, 路, , 小英, 總統, , 蔡英文, ,...","[小英, 總統, 選對, 小英, 總統, 蔡英文, Tsai, Ing, wen, 彰化市,..."
3278,黃秀芳,【水牛伯】立法院長游錫堃 感謝立法院長 游游錫堃傍晚到彰化芬園 好好漢將軍廟（芬園。社口）為...,"[【, 水牛, 伯, 】, 立法院, 長, 游錫堃, , 感謝, 立法院, 長, , 游...","[水牛, 伯, 立法院, 長, 游錫堃, , 感謝, 立法院, 長, , 游游錫堃, 傍...","[水牛, 立法院, 游錫堃, 感謝, 立法院, 游游錫堃, 傍晚, 彰化, 芬園, 好好, ..."


In [27]:
df_unnested = df.explode('cleaned_removePunc').reset_index(drop=True)
print(len(df_unnested))
df_unnested.head(1)


467904


,PageName,Message,tokenized,cleaned,cleaned_removePunc
0,伍麗華｜Saidai / Reseres,天啊，竟然有這樣的結果？！ 眾目睽睽下，進行不記名投票，完全輕看在地居民，真是不可思議！ 今...,"[天, 啊, ，, 竟然, 有, 這樣, 的, 結果, ？, ！, , 眾目, 睽睽, 下...","[天, 竟然, , 眾目, 睽睽, 下, 不, 記名, 投票, 輕看, 居民, 思議, ...",竟然


In [31]:
df_unnested.to_pickle("./芷瑩候選人/bak/df_unnested.pkl")

In [32]:
df.to_pickle("./芷瑩候選人/bak/df.pkl")

In [33]:
df = pd.read_pickle("./芷瑩候選人/bak/df.pkl")

# 斷詞 用空格格開示範

In [127]:
docs = list(df['cleaned_removePunc'])
new_docs = []
for doc in docs:
    
    #print(doc)
    new_token_doc = ""
    for t in range(0, len(doc)):
        new_token_doc += doc[t]
        if t == len(doc)-1:  break
        new_token_doc += " "
    new_docs.append(new_token_doc)
docs = new_docs

In [133]:
docs[20000:20020]


['醫師 難為 白牌 計程 車司 機染 新冠肺炎 死亡 一家 診所 就診 外界 質疑 醫師 未在 第一 時間 通報 有人 當場 質問 診所 護理師 醫師 看病 但事 實上 白牌 計程 車司 機是 這名 醫師 發現 疑因 流感 症狀 建議 轉診 醫院 被驗 染上 新冠肺炎 醫師 女兒 臉書貼 文稱 爸爸 診兢兢業業 心目 父親 醫師 點評 一個 基層 醫師 角度 醫師 這位 前輩 並無 不妥 有警 覺性 懷疑 流感 重症 簡單 事實 感冒 流感 新冠肺炎 發病 初期 表現 難以 區分 問旅遊史 之外 沒轍 問接 觸史 司機載 過什麼 病人 恐怕 診所 做新冠 病毒 篩檢 不曉得 鍵盤 專家 批評 換他們 來當 醫師 肉眼 新冠肺炎 醫師 診所 看診 病人 來來 去去 我會 新冠肺炎 看不出 堅守崗位 盡力',
 '一句 形容 新型 冠狀 病毒 抗體 感冒 病毒 小看 感冒 病毒 當我們 身上 抗體 傷力 很強 哪一種 對付 感冒 病毒 拿手 兒童 兒童 可應付 13 病毒感染 箇中 高手 未來 碰到 孩子 感冒 生病 爸媽 正向 思考 累積 抗病毒 經驗值 老人 雖然身 經百戰 人生 哪種 病毒 沒見識過 厲害 碰到 全新 病毒 實在 應付 不來 阿公 阿嬤 在生活中 事物 更長 時間 學習 兒童 防疫 重點 保護 家中 慢性病 爸爸 男性 重症 防疫 恐慌 包容 黃瑽寧 武漢肺炎 2019 新型 冠狀 病毒',
 '21 星期五 早上 門診 掛號 12 00',
 '設計 時空 背景 今晚 台灣 醫師 這兩天 有流 鼻水 發燒 症狀 武漢肺炎 出國 接觸 過疫區 回來 案例 一開始 詢問 出國 沒接 觸史 可能性 不高 鼻水 感冒 病毒 機率 武漢肺炎 辦法 台灣 平均 至少 10 萬人會 感冒 鼻水 現階段 鼻水 當成 武漢肺炎 去驗 誤診 病毒 誤診 辦法 鼻水 輕症 本來 漏掉 再來 感冒 時本來 就要 保持良好 鼻衛生 習慣 在家 休養 感冒 病毒 傳給 別人 怪怪的',
 '簡單 張圖',
 '各部 配合 大海 撈針 破案 能力 有如 福爾 摩斯 令人 台灣 超強 查出 司機 感染 其邁 公布 答案 LINETODAY',
 '精華版 下次 全民 示範 正確 洗手 洗手 洗手 洗手 真的',
 '分享 希望 孩子 有四點 記得 我家 孩子 弄翻 東西 弟弟

In [129]:
len(docs)

59988

In [2]:
import pandas as pd

In [3]:
t = pd.read_pickle("../bak/df_unnested.pkl")

In [5]:
t.to_csv("../data/final/2017_23-doctors_clean_unnested.csv")